In [1]:
"""
Import necessary libraries
"""
import pandas as pd
import numpy as np
import matplotlib_inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

In [2]:
"""
Read in the data
"""
opportunities = pd.read_excel('Opportunity.xlsx')
NAICS = pd.read_excel('NAICS.xlsx')

/Users/benkarabinus/.local/share/virtualenvs/TOOL1_FINAL_PROJECT-_N6x3YZ3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Exploratory Data Analysis

## *Data Cleaning and Feature Engineering*

### Objectives
- Column naming convention
- NAICS categories must be joined with opportunities
- System close date should be subtracted from record creation date to create a new feature representing days an opportunity is active 
- Remove non-informative data
- Handle missing data
- Separate categorical and continuous features for further analysis

In [3]:
opportunities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3894 entries, 0 to 3893
Columns: 112 entries, (Do Not Modify) Opportunity to PFY RSM Services Spend (Account) (Account)
dtypes: datetime64[ns](12), float64(29), int64(1), object(70)
memory usage: 3.3+ MB


In [4]:
#opportunities.isnull().sum()[0:26]

In [5]:
#opportunities.isnull().sum()[26:51]

In [6]:
#opportunities.isnull().sum()[51:76]

In [7]:
#opportunities.isnull().sum()[76:101]

In [8]:
#opportunities.isnull().sum()[101:120]

*Rename columns for ease of use and readability*

In [9]:
opportunities.columns = opportunities.columns.str.replace('\([^)]*\)', '', regex=True)
opportunities.columns = opportunities.columns.str.replace('\.', '', regex=True)
opportunities.columns = opportunities.columns.str.replace(':', '')
opportunities.columns = opportunities.columns.str.replace('/', '_')
opportunities = opportunities.rename(columns=lambda x : x.strip())
opportunities.columns = opportunities.columns.str.replace(' ','_')
opportunities.columns = opportunities.columns.str.replace('Address_1', '')
opportunities.columns = opportunities.columns.str.replace('^_', '', regex=True)

In [10]:
# rename system systen generated GUID to opportunityID
opportunities.rename(columns={opportunities.columns[0]: 'opportunity_id', 'Status' : 'Won'},
                     inplace=True)
print(opportunities.columns[0])

opportunity_id


In [ ]:
"""
Sanity check
"""
for column in list(opportunities.columns):
    print(column)

In [11]:
opportunities['NAICS'] = opportunities['NAICS'].str.replace(r'\D', '')
opportunities['NAICS'] = opportunities['NAICS'].str.strip()

/var/folders/s4/yckpqxmx6cnf837zqjqn_hxc0000gn/T/ipykernel_39628/389924680.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  opportunities['NAICS'] = opportunities['NAICS'].str.replace(r'\D', '')


In [12]:
opportunities['NAICS'] = opportunities['NAICS'].fillna(value=0)
opportunities['NAICS'].isnull().sum()
opportunities['NAICS'] = opportunities['NAICS'].astype(int)

In [13]:
opportunities = pd.merge(opportunities, NAICS, how='inner', left_on = ['NAICS'],  right_on = ['mcg_naicscode'])

*Handle Missing and Non-informative Data*

- Drop columns containing non-informative data
- Drop all features missing 50% or more data
- Replace missing values for categorical features with "\_missing_"
- Replace missing numerical features with appropriate values or drop if necessary

In [14]:
opportunities.shape

(3451, 119)

In [ ]:
# drop unecessary columns
opportunities.drop(columns=['Row_Checksum', 'LOB_Service_Line', 'Solution_Set', 'D&B_Company', 
                            'D&B_Financial', 'IPM_Date_Acquired', 'IPM_Date_Lost', 'Import_Date', 
                            'Record_Created_On', 'Modified_On', 'Actual_Close_Date', 'Sales_Stage', 
                            'Original_Opportunity', 'D&B_Company', 'Assets','Est._Fees', 'Total_Amount',
                            'Revenue_This_Year_Aggregated', 'Status_Reason', 'MSA', 'Rating','Consulting_Lead',
                            'Region', 'Deliverability_Rating', 'Relationship_Strength', 'mcg_naicscode',
                            'mcg_name', 'mcg_sicdescription', 'Commissonable', 'CJP_Target', 'AccountOriginal_Source', 
                            'Business_Type','D&B_NAICS', 'National_Initiative', 'Territory_Code', 'Last_Interaction',
                            'NAICS', 'Created_By'], axis=1, inplace=True)

print("Dropping non-informative features reduces the dimension of the dataset to {} features.".format(len(opportunities.columns)))

In [ ]:
for i in range(len(opportunities.columns)):
    print(i, opportunities.columns[i])

In [ ]:
opportunities.rename(columns={'mcg_industry_displayname' : 'industry','mcg_industrysector_displayname' : 'industry_sector', 
                                'mcg_subsectortext0' : 'industry_subsector', 'mcg_description' : 'description'}, inplace=True)

In [ ]:
print(opportunities['Created_On'].dtype)
print(opportunities['Est._Close_Date'].dtype)
print(opportunities['System_Close_Date'].dtype)

In [ ]:
opportunities['days_active'] = opportunities['System_Close_Date'] - opportunities['Created_On']
opportunities['days_active'] = opportunities['days_active'] / np.timedelta64(1, 'D')
opportunities['days_active'][opportunities['days_active'] < 0].count()

In [ ]:
problem_dates = \
opportunities[['days_active', 'Created_On', 'System_Close_Date']][opportunities['days_active'] < 0]

Creating the column "days_active" and counting days_active values < 0 shows a total of 858 opportunities that were active for a negative amount of days. This is likely due to errors during data entry or import of records into the customer relationship management system.

In [ ]:
opportunities[['opportunity_id', 'Entity_ID']].isna().sum()

In [ ]:
# drop columns missing more than 50% of values
perc = 0.50
threshold = int(perc*opportunities.shape[0])
opportunities.dropna(axis=1, thresh=threshold, inplace=True)
print("Dropping features missing 50 percent or more data reduces the dimension of the dataset"
" to {} features.".format(len(opportunities.columns)))

Separate continuous and categorical features

In [ ]:
# seperate numerical and categorical columns
opportunityID = opportunities[['opportunity_id', 'Entity_ID']].copy()
opp_numeric = opportunities.select_dtypes(include=[np.number])
opp_non_numeric = opportunities.select_dtypes(exclude=[np.number])
opp_non_numeric.drop(columns=['City','Street_1', 'State_Province', 'Country_Region','County',
                             'ZIP_Postal_Code'], inplace=True)
opp_geographic = opportunities.iloc[:, 0:10].copy()

In [ ]:
opp_numeric = pd.merge(opportunityID, opp_numeric, left_index=True, right_index=True, how='inner')
opp_non_numeric = pd.merge(opportunityID, opp_non_numeric, left_index=True, right_index=True, how='inner')
opp_numeric.drop(columns=['Entity_ID_y', 'Latitude', 'Longitude'], axis=1, inplace=True)
opp_non_numeric.drop(columns=['opportunity_id_y'], axis=1, inplace=True)
opp_numeric.rename(columns={'Entity_ID_x' : 'Entity_ID'}, inplace=True)
opp_non_numeric.rename(columns={'opportunity_id_x' : 'opportunity_id'}, inplace=True)
opp_numeric.set_index(['opportunity_id', 'Entity_ID'], inplace=True)
opp_non_numeric.set_index(['opportunity_id', 'Entity_ID'], inplace=True)
opp_geographic.set_index(['opportunity_id', 'Entity_ID'], inplace=True)

In [ ]:
# sanity check
print(opp_numeric.shape)
print(opp_non_numeric.shape)

In [ ]:
opp_non_numeric.isnull().sum()

In [ ]:
# fillna for categorical features with value "_missing_"
opp_non_numeric.fillna(value='_missing_', inplace=True)
opp_non_numeric.isnull().sum()

In [ ]:
opp_numeric.isnull().sum()

Printing a count of null values for each continuous variable shows that the columns "CFY_RSM_Services_Spend" and "PFY_RSM_Services_Spend" are both missing 453 observations each. It's assumed that missing values in these columns represent clients who have not purchased consulting services from the firm in either of the two periods. As such, missing values will be replaced with 0. Furthermore, each of the remaining continuous predictors missing data represent a small fraction of the total record count and should be dropped to improve model performance.

In [ ]:
# fill current fiscal year spend and previous fiscal year spend with 0, assuming if value not given no prior or current spend
opp_numeric['CFY_RSM_Services_Spend'] = opp_numeric['CFY_RSM_Services_Spend'].fillna(value=0)
opp_numeric['PFY_RSM_Services_Spend'] = opp_numeric['PFY_RSM_Services_Spend'].fillna(value=0)

In [ ]:
# Drop rows where annual revenue or number of employees is missing
opp_numeric = opp_numeric.dropna(subset=['Annual_Revenue', 'No._of_Employees'])

In [ ]:
# print dimensions
opp_numeric.shape

In [ ]:
# sanity check
opp_numeric.isnull().sum()

## *Explore Categorical Features*

Objectives
- Examine frequency distruibution of categorical features
- Drop further features based on domain knowledge

In [ ]:
"""Transform "Status" to binary outcome"""
opp_non_numeric['Won'] = opp_non_numeric['Won'].apply(lambda x : 1 if (x == 'Won') else 0)

In [ ]:
opp_non_numeric.value_counts('Won',normalize=True)

In [ ]:
opp_non_numeric.value_counts('Type', normalize=True)

In [ ]:
opp_non_numeric.value_counts('Client_Status', normalize=True)

The features "Type" and "Client_Status" provide similar infomation abut a given opportunity. Type is a more informative and more complete feature. Client_Status will be dropped.

In [ ]:
opp_non_numeric.drop(columns=['Client_Status'], inplace=True)

In [ ]:
opp_non_numeric.value_counts('Currency')

"Currency" takes on a singe value "US Dollar." The feature adds no valuable information about the data.

In [ ]:
opp_non_numeric.drop(columns=['Currency'], inplace=True)

In [ ]:
opp_non_numeric.value_counts('Lead_Source', normalize=True)

In [ ]:
opp_non_numeric.value_counts('Preferred_Method_of_Contact')

"Preferred_Method_of_Contact" takes ona single value any. The feature provides no additional information about the dataset.

In [ ]:
opp_non_numeric.drop(columns=['Preferred_Method_of_Contact'], inplace=True)

In [ ]:
opp_non_numeric.value_counts("VIP_Client?")

"VIP_Client?" takes on a single value "No." The feature provides no additional information about the dataset

In [ ]:
opp_non_numeric.drop(columns=['VIP_Client?'], inplace=True)

In [ ]:
opp_non_numeric.value_counts('Microsoft_Enterprise_Agreement')

In [ ]:
fig = px.treemap(opp_non_numeric, path=[px.Constant('all'), 'industry', 'industry_sector'], 
                 values = 'Won', title='Wins by Industry Sector')
fig.update_layout(title_x=0.5, margin = dict(t=35, l=25, r=25, b=25))
fig.show()

### Methods

- Print Summary Statistics
- Inspect Data Distribution

In [ ]:
"""Print summary statistics"""
pd.set_option('display.float_format', lambda x : '%.2f' % x)
opp_numeric.describe()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20, 15))
sns.histplot(opp_numeric, x='Est._Revenue', stat='density', kde=True, ax=ax[0,0])
sns.histplot(opp_numeric, x='No._of_Employees', stat='density', kde=True, ax=ax[0, 1])
sns.histplot(opp_numeric, x='Open_Revenue', stat='density', kde=True, ax=ax[0, 2])
sns.histplot(opp_numeric, x='Annual_Revenue', stat='density', kde=True, ax=ax[1,0])
sns.histplot(opp_numeric, x='CFY_RSM_Services_Spend', stat='density', kde=True, ax=ax[1, 1])
sns.histplot(opp_numeric, x='PFY_RSM_Services_Spend', stat='density', kde=True, ax=ax[1, 2])
plt.ticklabel_format(style='Plain')
fig.suptitle('Distribution of Continuous Variables (High-Level)')
plt.tight_layout()

## *Explore Relationship Between Variables*

### Methods

- Pearson's Correlation
- Distribution Plots

In [ ]:
# plot correlation of numeric data
sns.set(rc={"figure.figsize":(10, 6)})
correlation = sns.heatmap(opp_numeric.corr(), cmap='rocket',
                          annot=True).set_title('Pearson\'s Correlation')

In [ ]:
fig = px.histogram(opp_numeric, x='Est._Revenue')
fig.show()

In [ ]:
fig = px.box(opp_numeric, y='No._of_Employees')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=x0))
fig.add_trace(go.Histogram(x=x1))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
opportunities = pd.merge(opp_numeric, opp_non_numeric, left_index=True, right_index=True, how='inner')
#opportunities = opp_numeric.join(opp_non_numeric, lsuffix='left', rsuffix='right')
for i in range(len(opportunities.columns)):
    print(i, opportunities.columns[i])

In [ ]:
opp_non_numeric.shape

In [ ]:
opportunities.isnull().sum()

In [ ]:
opportunities.to_excel('Opportunity_reduced.xlsx')